## To run test 44 +. Till test 6 was run by Amit Pandey ada account. Test 7- 20 was run from dhawal : wmd_min_re_test7 notebook. , then test 21 - 32 using wmd_min_test21, then test33-43 in WMD_min_RE_for_loop_test33_Dhawal_scratch_v1

        ## result file = result_dhawal_test21

In [2]:
## To run as batch job


#imports:

# file imports

import numpy as np
import matplotlib.pyplot as plt
import gensim
import gensim.downloader
import os
from scipy.optimize import linprog
import nltk
from collections import defaultdict
from gensim.models import KeyedVectors
import sklearn
import scipy
import time
from multiprocessing import Pool
from nltk.corpus import stopwords
import json
from multiprocessing import Pool
import time
import pickle
from gensim import models

#nltk.download('stopwords')
#nltk.download('punkt')
# nltk.download('wordnet')

print("code started and imports done \n")




print("\n Directory check :\n")
print("\n /scratch/Amit_Pandey/wmd_lite/files:", os.listdir("/scratch/Amit_Pandey/wmd_lite/files"))
print(" \n os.listdir /scratch/Amit_Pandey/gensim-data/word2vec-google-news-300/:",
      os.listdir("/scratch/Amit_Pandey/gensim-data/word2vec-google-news-300/"))

print("\n printing current nltk path and adding to the path:")
print(nltk.data.path)

nltk.data.path.append("/scratch/Amit_Pandey/nltk_data")

print("\n",nltk.data.path) 




def sentence_preprocess(embed_dict, sentence,lowercase = 1, strip_punctuation = 1,  remove_stopwords = 1,removedigit = 1):
    ''' 1 : True, 0 : False : Lowercase, Strip puncutation, Remove Stopwords, removedigit'''

    stop_words = list(stopwords.words('english'))

    if lowercase == 1:
        sentence = sentence.lower()

    sentence_words = nltk.word_tokenize(sentence)

    if strip_punctuation == 1 and removedigit == 1:
        sentence_words = [word for word in sentence_words if word.isalpha()] 
        


    if remove_stopwords == 1:
        sentence_words = [word for word in sentence_words if not word in stop_words]
    
    ## to remove those words which are not in the embeddings that we have.
    
    sentence_words = [word for word in sentence_words if word in embed_dict.keys()]



    return sentence_words




embeddingtype = None
embd_model = None



## to load from embedding text files:
## have used this to load glove vectors and not word2vec

def load_glove(embeddingtype):
    
    if embeddingtype == 3:
        i = 300
        
        a_file = open("/scratch/Amit_Pandey/wmd_lite/files/reduced_glove_embedding_300.json", "r")
        output = json.load(a_file)
        print(len(output.keys()))
        a_file.close()
        
        embeddings_dict = output
        
    if embeddingtype == 4:
        i = 200
    if embeddingtype == 5:
        i = 100
    if embeddingtype == 6:
        i = 50
    
    
#     embeddings_dict = defaultdict(lambda:np.zeros(i)) 
#     # defaultdict to take care of OOV words.
    
#     with open(f"../files/glove.6B.{i}d.txt",'r') as f:
#         for line in f:
#             values = line.split()
#             word = values[0]
#             vector = np.asarray(values[1:], "float32")
#             embeddings_dict[word] = vector
        
    return embeddings_dict



def embeddings_setup(newembeddingtype):
    
    
    global embeddingtype
    global embd_model
    
    
    '''to avoid loading all the embeddings in the memory.'''
    
    ''''## Note : we are finding the embd matrix two times, ie once for each sentence in
        ## the pair of sentences.
        ## so this happens that embedding type is changed when find_embmatrix is called
        ## by the first sentence.
        The above line doesnt matter now as we not calling find_embmatrix , instead we setting up.
    '''
        
        
        
    if ( embeddingtype != newembeddingtype):
        #print("embdtype  entered :", embeddingtype != newembtype,"\n")
        #print("embd_model type changed to :", type(embd_model),"\n" )
        
        embeddingtype = newembeddingtype
        
        #embd_model = embeddings_setup(embeddingtype) #adictionary
        
        #print("embd_model type changed to :", type(embd_model),"\n" )
        #to make sure that we don't download the embeddings again and again,
        # we will check if the embedding type is same as the old one
        # and update global embd_model, vrna next time vo use hi nhi ho payega.
    
    
    
    
    
    if embeddingtype == 1:
        
        ## To load from scratch:
        
        w = models.KeyedVectors.load_word2vec_format(
        '/scratch/Amit_Pandey/gensim-data/word2vec-google-news-300/GoogleNews-vectors-negative300.bin', binary=True)
        
        embedding = w
        
        #embedding = KeyedVectors.load('google300w2v.kv', mmap='r')
        ## This will be slower but will prevent kernel from crashing.
        
        ## comment the above line and uncomment this if you have sufficient RAM:
        
        #w2v_emb = gensim.downloader.load('word2vec-google-news-300')
        
    if embeddingtype == 2:
        print('Normalised word2vec not loaded, will get it soon')
        embedding = None
    
    if embeddingtype in (3,4,5,6):
        embedding = load_glove(embeddingtype)
        
    
    embd_model = embedding
    
    
        
def find_embdMatrix(sentence):
    global embeddingtype
    global embd_model
    #print(" global embedding type being passed is :", embeddingtype,"\n")
    #print("embedding type received by the find emb matrix is :", newembtype,"\n")
    #print("embd model type is :", type(embd_model),"\n")
    
    sent_mtx = []
    
    
    ##commented lines moved to embedding setup.
    
#     ''''## Note : we are finding the embd matrix two times, ie once for each sentence in
#     ## the pair of sentences.
#     ## so this happens that embedding type is changed when find_embmatrix is called
#     ## by the first sentence
#     '''
#     if ( embeddingtype != newembtype):
#         print("if embdtype part entered :", embeddingtype != newembtype,"\n")
        
#         embeddingtype = newembtype
#         embd_model = embeddings_setup(embeddingtype) #adictionary
        
#         print("embd_model type changed to :", type(embd_model),"\n" )
#     #to make sure that we don't download the embeddings again and again,
#     # we will check if the embedding type is same as the old one
#     # and update global embd_model, vrna next time vo use hi nhi ho payega.
    
    #print("embd_model type changed to :", type(embd_model),"\n" )
    for word in sentence:
        word_emb = embd_model[word]
        sent_mtx.append(word_emb)
    
    sent_mtx = np.array(sent_mtx).reshape(len(sentence),-1)

    return sent_mtx




def wasserstein_distance(pi, qj, D, cost = 'min'):
        """Find Wasserstein distance through linear programming
        p.shape=[m], q.shape=[n], D.shape=[m, n]
    
        suppose doc1 has m words and doc2 has n words, then an mxn array would be formed, 
        having distance of each word in doc1 to that of doc2.
    
    
    
        p.sum()=1, q.sum()=1, p∈[0,1], q∈[0,1]
        """
        A_eq = [] # a list which will later be converted to array after appending.
        for i in range(len(pi)): # len = number of words.
            A = np.zeros_like(D) # a 2d array made with the shape of D.  
            A[i, :] = 1 
            #print("Dshape, len pi till here :",D.shape,len(pi),"\n")
            
            # to make summation over "i" of Tij = pi, ie total / sum of outflow
            ## from one word is equal to its pi (normalized bag of word/ frequency/density)
            ## ex : if 2x3 D:
            ##T1,1 + T1,2 + T1,3 + 0 T2,1 + 0 T2,2 + 0 T2,3 = P1 and so on for every i,
            ## ie for each word in the doc1
            
            
            #print("A.shape", A.shape,"\n")
            A_eq.append(A.reshape(-1)) ## reshape(-1) flatens and then appending in A_eq.
            
            #print(A_eq,"Aeq\n")
            
            
            
            ## A_eq will be (m+n)x(m.n)
    
        for i in range(len(qj)):
            A = np.zeros_like(D)
            A[:, i] = 1 ## summation over "j" this time, so this time for different rows, 
            ## over a column "j" which refers to doc2, ie total incoming flow = qj density
            A_eq = list(A_eq)
            A_eq.append(A.reshape(-1))
            A_eq = np.array(A_eq)
        
        #print(A_eq.shape,A_eq)
       
        b_eq = np.concatenate([pi, qj])
        D = D.reshape(-1)
        #print("Dshape:",D.shape)
        if cost == 'max':
            D = D*(-1)
        
        result = linprog(D, A_eq=A_eq[:-1], b_eq=b_eq[:-1]) ## removing redundant to make 
        ## solution more robust.
        return np.absolute(result.fun), result.x , D.reshape((len(pi),len(qj)))  ## fun returns the final optimized value, x returns each value of xi,j that is the array

    
def relaxed_distance(pi,qj,D,cost='min'):
    
    # to find relaxed we just add the min/max cost directly using the least distance for pi to qj.
    
    # D is calculated from P to Q ie P in rows and Q in columns, To find Q to P we will transpose 
    if cost == 'min':
        p_to_q = np.dot(D.min(axis=1),pi)
        q_to_p = np.dot(D.T.min(axis=1),qj)
        
        return max(p_to_q,q_to_p)
    
    if cost == 'max':
        
        p_to_q = np.dot(D.max(axis=1),pi)
        q_to_p = np.dot(D.T.max(axis=1),qj)
        
        return min(p_to_q,q_to_p), None, D
        
        
    
    
class WMD:
    
    ''' wmd type = normal/relaxed, costtype = min/max.
    Enter Two sentence strings, cost = max if you want to try 
    max cost max flow version, embeddingtype = 1 for word2vec, 2 = normalized
    word2vec, 3 = glove300d, 4 = glove200d, 5 = glove100d 6 = glove50d'''
    
    def __init__(self,embeddingtype, wmd_type = 'normal', costtype='min'):
        
        
        self.cost = costtype
        
        self.embeddingtype = embeddingtype 
        self.wmd_type = wmd_type
        
        
        ## setting up the embeddings
        
        embeddings_setup(self.embeddingtype)
        
        
        
        
    #def word_count(self):
#         self.sent1_dic = defaultdict(int)
#         self.sent2_dic = defaultdict(int)
        
#         for word in sorted(sentence_preprocess(self.sent1)):
#             self.sent1_dic[word] += 1
            
#         for word in sorted(sentence_preprocess(self.sent2)):
#             self.sent2_dic[word] += 1
        
#         return dict(self.sent1_dic), dict(self.sent2_dic)



#     def wasserstein_distance(self, pi, qj, D):
#         """Find Wasserstein distance through linear programming
#         p.shape=[m], q.shape=[n], D.shape=[m, n]
    
#         suppose doc1 has m words and doc2 has n words, then an mxn array would be formed, 
#         having distance of each word in doc1 to that of doc2.
    
    
    
#         p.sum()=1, q.sum()=1, p∈[0,1], q∈[0,1]
#         """
#         A_eq = [] # a list which will later be converted to array after appending.
#         for i in range(len(pi)): # len = number of words.
#             A = np.zeros_like(D) # a 2d array made with the shape of D.  
#             A[i, :] = 1 
#             # to make summation over "i" of Tij = pi, ie total / sum of outflow
            ## from one word is equal to its pi (normalized bag of word/ frequency/density)
            ## ex : if 2x3 D:
            ##T1,1 + T1,2 + T1,3 + 0 T2,1 + 0 T2,2 + 0 T2,3 = P1 and so on for every i,
            ## ie for each word in the doc1
        
#             A_eq.append(A.reshape(-1)) ## reshape(-1) flatens and then appending in A_eq.
            ## A_eq will be (m+n)x(m.n)
    
#         for i in range(len(qj)):
#             A = np.zeros_like(D)
#             A[:, i] = 1 ## summation over "j" this time, so this time for different rows, 
#             ## over a column "j" which refers to doc2, ie total incoming flow = qj density
#             A_eq.append(A.reshape(-1))
#             A_eq = np.array(A_eq)
        
#         print(A_eq.shape,A_eq)
       
#         b_eq = np.concatenate([pi, qj])
#         D = D.reshape(-1)
#         if self.cost == 'max':
#             D = D*(-1)
        
#         result = linprog(D, A_eq=A_eq[:-1], b_eq=b_eq[:-1]) ## removing redundant to make 
#         ## solution more robust.
#         return result.fun, result.x  ## fun returns the final optimized value, x returns each value of xi,j that is the array

    
    def word_mover_distance(self,sentence1,sentence2):
        
        self.sent1 = sentence1
        #print(self.sent1 ,"\n")
        self.sent2 = sentence2
        #print(self.sent2 ,"\n")
        
        
        self.sent1_dic = defaultdict(int)
        self.sent2_dic = defaultdict(int)
        
        for word in sorted(sentence_preprocess(embd_model,self.sent1)): # sorted to have better
            self.sent1_dic[word] += 1 # idea of the sequence of the words. Creating BOW here
            
        for word in sorted(sentence_preprocess(embd_model,self.sent2)): #creating BOW from sorted sequence
            self.sent2_dic[word] += 1
        
        
        self.sent1_dic = dict(self.sent1_dic) # converted from default dict to dict.
        self.sent2_dic = dict(self.sent2_dic) # because following operations work on dict
        
        
        #print(self.sent1_dic ,"\n")
        #print(self.sent2_dic ,"\n")
        
        
        ## Now we will store a list/array of all the words in each sentence (in alphabetically sorted order)
        ## we will store corresponding count, and then corresponding Normalised count.
        self.sent1_words = np.array(list(self.sent1_dic.keys())) #dictionary keys converted to list than array
        self.sent1_counts = np.array(list(self.sent1_dic.values()))
        
        self.sent2_words = np.array(list(self.sent2_dic.keys()))
        self.sent2_counts = np.array(list(self.sent2_dic.values()))
        
        
        #print(self.sent1_words ,"\n")
        #print(self.sent1_counts ,"\n")
        
        #print(self.sent2_words ,"\n")
        #print(self.sent2_counts ,"\n")
        
        #dictionary values cant be converted into an array directly, hence the
        #list step.
        
        #print("embedding type being passed is :", self.embeddingtype,"\n")
        self.sent1_embmtx = find_embdMatrix(self.sent1_words)
        #print(self.sent1_embmtx.shape,"sent1emb\n")
        self.sent2_embmtx = find_embdMatrix(self.sent2_words)
        #print(self.sent2_embmtx.shape,"sent2emb\n")
        
        self.pi = self.sent1_counts/np.sum(self.sent1_counts) #NBOW step from BOW
        #print(self.pi,"self.pi\n")
        self.qj = self.sent2_counts/np.sum(self.sent2_counts)
        #print(self.qj,"self.qj\n")
        
        self.D = np.sqrt(np.square(self.sent1_embmtx[:, None] - self.sent2_embmtx[None, :]).sum(axis=2)) 
        #print(self.D.shape,"Dshape \n")
        ## programmers sought used mean instead of sum.
        ## scipy cdist can be used as well.
        
        if self.wmd_type == 'normal':
            return wasserstein_distance(self.pi, self.qj, self.D, self.cost)
        
        
        if self.wmd_type == 'relaxed':
            return relaxed_distance(self.pi,self.qj,self.D,self.cost)
        
print("\n FUNCTIONS DEFINITION OVER AND DATA LOADING STARTED\n")


## KNN

Train_BBCsport_sent = np.load("/scratch/Amit_Pandey/wmd_lite/files/Train_BBCsport_sent.npy")
Train_BBCsport_label = np.load("/scratch/Amit_Pandey/wmd_lite/files/Train_BBCsport_label.npy")
Test_BBCsport_sent = np.load("/scratch/Amit_Pandey/wmd_lite/files/Test_BBCsport_sent.npy")
Test_BBCsport_label = np.load("/scratch/Amit_Pandey/wmd_lite/files/Test_BBCsport_label.npy")


print("\n DATA LOADING ENDED\n")

#for i in range(5):
    #print(Test_BBCsport_label[i],"\n",Test_BBCsport_sent[i])
    

print("##################Train details:\n")

#for i in range(5):
    #print(Train_BBCsport_label[i],"\n",Train_BBCsport_sent[i])


# embeddingtype = 3
# model = WMD(embeddingtype,wmd_type = 'relaxed', costtype='max')

            
    
no_testdocs = len(Test_BBCsport_sent)
no_testlabels = len(Test_BBCsport_label)
#no_testdocs,no_testlabels



actual_category = []
predicted_category = []

    



#import time
st = time.time()
print("\n MODEL INITIALIZATION STARTED\n")
embeddingtype = 3
model = WMD(embeddingtype,wmd_type = 'normal', costtype='min')

print("\n MODEL INITIALIZATION OVER\n")

result_Dhawal_test44 = []
test_finished_Dhawal_test44 = []


def predict_Category(i):
    global result_Dhawal_test44 
    global test_finished_Dhawal_test44
    
    prediction_dictionary = {}
    sentence = Test_BBCsport_sent[i]
    
    distance_fromTrainset = []
    
    for j in range (len(Train_BBCsport_sent)):
    #for j in range (10): #number of train
        ## Find totalcost ie distance between sentence passed from test set to each sentence 
        ## in training set. and then append in the list.
        
        #print(sentence)
        #print(Train_BBCsport_sent[i])
        
        print(f"\nTrain{j}")
        
        Totalcost, Tcoeff, Distancematx = model.word_mover_distance(sentence,Train_BBCsport_sent[j])
        print(f" distance btwn test{i} and train{j} :", Totalcost,"\n")
        #print(Totalcost)
        distance_fromTrainset.append(Totalcost)
        
    distance_fromTrainset = np.array(distance_fromTrainset)
    #print('distance from train set array:',distance_fromTrainset)
    
    arr1indx = distance_fromTrainset.argsort()
    print("index of distance in increasing order is:", arr1indx, "\n")
    
    #print("Distance and label sorted from test set",distance_fromTrainset[arr1indx[::1]], "\n",Train_BBCsport_label[arr1indx[::1]],"\n","Sentences: \n",Train_BBCsport_sent[arr1indx[::1]]) 
    
    
    ## Taking for different values of K
    
    #k = 5
    sorted_distance_fromTrainset_k5 = distance_fromTrainset[arr1indx[::1]][:5]
    sorted_labels_k5 = Train_BBCsport_label[arr1indx[::1]][:5]
    
    predicted_cat_k5 = scipy.stats.mode(sorted_labels_k5)[0]
    #print("pred 5",predicted_cat_k5)
   

    #k = 7
    sorted_distance_fromTrainset_k7 = distance_fromTrainset[arr1indx[::1]][:7]
    sorted_labels_k7 = Train_BBCsport_label[arr1indx[::1]][:7]
    
    predicted_cat_k7 = scipy.stats.mode(sorted_labels_k7)[0]
    #print("pred 7",predicted_cat_k7)

    #k = 11
    sorted_distance_fromTrainset_k11 = distance_fromTrainset[arr1indx[::1]][:11]
    sorted_labels_k11 = Train_BBCsport_label[arr1indx[::1]][:11]
    
    predicted_cat_k11 = scipy.stats.mode(sorted_labels_k11)[0]
    #print("pred 11",predicted_cat_k11)

    #k = 15
    sorted_distance_fromTrainset_k15 = distance_fromTrainset[arr1indx[::1]][:15]
    sorted_labels_k15 = Train_BBCsport_label[arr1indx[::1]][:15]
    
    predicted_cat_k15 = scipy.stats.mode(sorted_labels_k15)[0]
    #print("pred 15",predicted_cat_k15)

    #k = 21
    sorted_distance_fromTrainset_k21 = distance_fromTrainset[arr1indx[::1]][:21]
    sorted_labels_k21 = Train_BBCsport_label[arr1indx[::1]][:21]
    
    predicted_cat_k21 = scipy.stats.mode(sorted_labels_k21)[0]
    #print("pred 21",predicted_cat_k21)


    #print(sorted_distance_fromTrainset,sorted_labels)
    prediction_dictionary[i] = [Test_BBCsport_label[i],
                                arr1indx[:30].tolist(),
                                Train_BBCsport_label[arr1indx[::1]][:30].tolist(),
                                distance_fromTrainset[arr1indx[::1]][:30].tolist(),
                                [predicted_cat_k5.tolist(),predicted_cat_k7.tolist(), predicted_cat_k11.tolist(),predicted_cat_k15.tolist(),
                                 predicted_cat_k21.tolist()]]
    
    
    result_Dhawal_test44.append(prediction_dictionary)
    test_finished_Dhawal_test44.append(i)
    
    
    with open('../results/result_Dhawal_frominteractive_test44.pickle', 'wb') as handle:
        pickle.dump(result_Dhawal_test44, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    with open('../results/test_finished_Dhawal_frominteractive_test44.pickle', 'wb') as handle:
        pickle.dump(test_finished_Dhawal_test44, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    
    
    
    
    #return np.array([predicted_cat_k5,predicted_cat_k7,predicted_cat_k11,predicted_cat_k15,predicted_cat_k21])   
        
    
    
    
no_testdocs = len(Test_BBCsport_sent)
no_testlabels = len(Test_BBCsport_label)
#no_testdocs,no_testlabels



actual_categories = []
predicted_categories_list = []
for i in range (44,50): # number of test
    print(f" \n ################### Test{i} #############","\n")
    predict_Category(i)
    
    #actual_categories.append(Test_BBCsport_label[i]) 
    #pred_category = predict_Category(Test_BBCsport_sent[i])
    #print(pred_category)
    #predicted_categories_list.append(pred_category)
    

et = time.time() 

print("time taken:",et-st)


    



   
 


code started and imports done 


 Directory check :


 /scratch/Amit_Pandey/wmd_lite/files: ['Test_BBCsport_sent.npy', 'bbcsport_dataset.json', 'bbcsport_labels.npy', 'data.json', 'model.npy', 'Train_BBCsport_label.npy', 'Train_BBCsport_sent.npy', 'glove.6B.zip', 'wmd_exp_v4_for_loop_RE_for_Dhawal.ipynb', 'bbcsport', 'model.npz', 'glove.6B.50d.txt', 'reduced_glove_embedding_300.json', 'google300w2v.kv', 'glove.6B.200d.txt', 'bbcsport_sentences.npy', 'Test_BBCsport_label.npy', 'glove.6B.100d.txt', 'glove.6B.300d.txt']
 
 os.listdir /scratch/Amit_Pandey/gensim-data/word2vec-google-news-300/: ['GoogleNews-vectors-negative300.bin', '__pycache__', 'word2vec-google-news-300.gz', '__init__.py']

 printing current nltk path and adding to the path:
['/home2/dhawals1939/nltk_data', '/home2/dhawals1939/miniconda3/nltk_data', '/home2/dhawals1939/miniconda3/share/nltk_data', '/home2/dhawals1939/miniconda3/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/

 distance btwn test44 and train102 : 6.254356047472216 


Train103
 distance btwn test44 and train103 : 5.800700618419461 


Train104
 distance btwn test44 and train104 : 6.389005954076594 


Train105
 distance btwn test44 and train105 : 6.841540504553674 


Train106
 distance btwn test44 and train106 : 6.45134626306022 


Train107
 distance btwn test44 and train107 : 6.223210749149959 


Train108
 distance btwn test44 and train108 : 5.656256901355877 


Train109
 distance btwn test44 and train109 : 6.452187359666569 


Train110
 distance btwn test44 and train110 : 6.5959143831717135 


Train111
 distance btwn test44 and train111 : 6.110389853344067 


Train112
 distance btwn test44 and train112 : 5.906561552488253 


Train113
 distance btwn test44 and train113 : 6.0373235422407845 


Train114
 distance btwn test44 and train114 : 6.376249307217957 


Train115
 distance btwn test44 and train115 : 7.179010857190376 


Train116
 distance btwn test44 and train116 : 6.262595412241034 


Tra

/tmp/ipykernel_5440/1081797254.py:275: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
  result = linprog(D, A_eq=A_eq[:-1], b_eq=b_eq[:-1]) ## removing redundant to make
/tmp/ipykernel_5440/1081797254.py:275: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  result = linprog(D, A_eq=A_eq[:-1], b_eq=b_eq[:-1]) ## removing redundant to make
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=6.25808e-23): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


 distance btwn test44 and train175 : 5.576423437502834 


Train176
 distance btwn test44 and train176 : 6.506837154447869 


Train177
 distance btwn test44 and train177 : 7.137929605510914 


Train178
 distance btwn test44 and train178 : 6.273389509343119 


Train179
 distance btwn test44 and train179 : 5.26098507403787 


Train180
 distance btwn test44 and train180 : 6.1388667372726635 


Train181
 distance btwn test44 and train181 : 6.126276728509016 


Train182
 distance btwn test44 and train182 : 6.2051276108858335 


Train183
 distance btwn test44 and train183 : 6.353766594721564 


Train184
 distance btwn test44 and train184 : 6.004221559377651 


Train185
 distance btwn test44 and train185 : 6.887141949497762 


Train186
 distance btwn test44 and train186 : 6.8786740431395295 


Train187
 distance btwn test44 and train187 : 5.911509174870709 


Train188
 distance btwn test44 and train188 : 6.304602425422936 


Train189
 distance btwn test44 and train189 : 7.108508247327617 


Tr

/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=2.22651e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


...

 distance btwn test48 and train511 : 5.571463898124082 


Train512
 distance btwn test48 and train512 : 5.524841973279232 


Train513
 distance btwn test48 and train513 : 6.414538152793686 

index of distance in increasing order is: [396 352 502 488  27 179  54 357 310 435  39 201 424 283 233 268 152 329
  38  80 399 135  79 175 477 121  65 505 413 412 512 229 291 320 402  88
 461 478 388 511 232 149 133 248 434 467 510 280  97 359 187 169 213 504
 120 506 507 334 492 261 103  42 126 270 293 319   5  99  31  13 191 305
 432  41  93 416   7 465 239 382 457  69 353  19 125 246 228 148  21  30
 118 314 362 315 440 407  20 245 299 326 108  74 453 153 304 323  45 318
 238 101 360 387 182 214  76 409 497 184  63 485  94 288 335 124 448 217
 243 128 475  92 438 137  68 466 474 327 480  17 139  64  75 307 192 102
 255   6 236 414 468 309 415 264 151 403  57 491 100  32   9 330 385 487
 392 112 379 113 503 344 163 162 447 244 188 431 322 391 204 212 493 469
  62 267 380 272 464 178 333 180 508 

 distance btwn test49 and train90 : 7.199864370120997 


Train91
 distance btwn test49 and train91 : 6.889292670286702 


Train92
 distance btwn test49 and train92 : 7.187419374878241 


Train93
 distance btwn test49 and train93 : 7.19284156496739 


Train94
 distance btwn test49 and train94 : 6.908240354526127 


Train95
 distance btwn test49 and train95 : 6.814443340870099 


Train96
 distance btwn test49 and train96 : 7.34899424317952 


Train97
 distance btwn test49 and train97 : 6.787344997519339 


Train98
 distance btwn test49 and train98 : 6.921269070805325 


Train99
 distance btwn test49 and train99 : 7.145719585815687 


Train100
 distance btwn test49 and train100 : 6.4653902676747865 


Train101
 distance btwn test49 and train101 : 6.878474668992606 


Train102
 distance btwn test49 and train102 : 7.212786147503763 


Train103
 distance btwn test49 and train103 : 6.562397282973809 


Train104
 distance btwn test49 and train104 : 6.683375092836131 


Train105
 distance btwn 

 distance btwn test49 and train213 : 7.327820929321626 


Train214
 distance btwn test49 and train214 : 7.042481897767393 


Train215
 distance btwn test49 and train215 : 6.238537901635013 


Train216
 distance btwn test49 and train216 : 7.007841353537857 


Train217
 distance btwn test49 and train217 : 7.241758561457142 


Train218
 distance btwn test49 and train218 : 7.222854369453124 


Train219
 distance btwn test49 and train219 : 6.029105786388757 


Train220
 distance btwn test49 and train220 : 7.297533360601196 


Train221
 distance btwn test49 and train221 : 7.23698465689929 


Train222
 distance btwn test49 and train222 : 7.256375951774677 


Train223
 distance btwn test49 and train223 : 7.0813252450767425 


Train224
 distance btwn test49 and train224 : 7.353827823911846 


Train225
 distance btwn test49 and train225 : 5.651453009017978 


Train226
 distance btwn test49 and train226 : 7.0865899704148205 


Train227
 distance btwn test49 and train227 : 6.75024065923896 


Trai

 distance btwn test49 and train336 : 7.084898970695122 


Train337
 distance btwn test49 and train337 : 7.377238263110051 


Train338
 distance btwn test49 and train338 : 7.7315596667762385 


Train339
 distance btwn test49 and train339 : 7.11700267545903 


Train340
 distance btwn test49 and train340 : 7.201287232713891 


Train341
 distance btwn test49 and train341 : 7.288154992823477 


Train342
 distance btwn test49 and train342 : 6.838211175025486 


Train343
 distance btwn test49 and train343 : 6.9474455476805606 


Train344
 distance btwn test49 and train344 : 6.880181644515442 


Train345
 distance btwn test49 and train345 : 7.3868122335894775 


Train346
 distance btwn test49 and train346 : 7.206519544480882 


Train347
 distance btwn test49 and train347 : 6.120341123355956 


Train348
 distance btwn test49 and train348 : 7.439530289081503 


Train349
 distance btwn test49 and train349 : 7.202674470475328 


Train350
 distance btwn test49 and train350 : 5.7679689506171545 


T

 distance btwn test49 and train459 : 7.557309245385243 


Train460
 distance btwn test49 and train460 : 7.195796338813423 


Train461
 distance btwn test49 and train461 : 6.755977219450202 


Train462
 distance btwn test49 and train462 : 7.239371684716547 


Train463
 distance btwn test49 and train463 : 7.308158778011899 


Train464
 distance btwn test49 and train464 : 6.829528926398703 


Train465
 distance btwn test49 and train465 : 6.81679401663755 


Train466
 distance btwn test49 and train466 : 6.971227745132724 


Train467
 distance btwn test49 and train467 : 6.958941371245418 


Train468
 distance btwn test49 and train468 : 7.0697243694854 


Train469
 distance btwn test49 and train469 : 6.316900656084577 


Train470
 distance btwn test49 and train470 : 6.442519720777625 


Train471
 distance btwn test49 and train471 : 7.7237131320126915 


Train472
 distance btwn test49 and train472 : 7.017070528461286 


Train473
 distance btwn test49 and train473 : 6.0544398577263605 


Train